In [27]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [29]:
subject = 'Paraíba - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [30]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Paraíba - Produção de Cimento (t)', axis=1)
data

,Ano,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH,Paraíba - Consumo de Cimento (t)
0,2008,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,0.618312,0.647756,5.483335e+08,2.546644e+09,1.874883e+10,...,15.141120,948.241851,1293.296773,22.416524,19.527739,85.157286,4.537486,0.776272,0.677959,707.783000
1,2009,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,0.622639,0.649827,5.302764e+08,2.679945e+09,2.055015e+10,...,15.401551,948.141401,1311.220354,22.466548,19.707229,85.814573,4.395830,0.778859,0.680849,734.672000
2,2010,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,0.626723,0.651346,5.046425e+08,2.891930e+09,2.342247e+10,...,15.187968,948.153818,1296.766840,22.336126,19.669010,86.268192,4.409472,0.780851,0.683306,834.169000
3,2011,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,0.630566,0.652325,4.854622e+08,3.091988e+09,2.604265e+10,...,14.484405,948.335984,1243.078346,22.321663,19.581222,85.940292,4.376316,0.782258,0.685344,946.248000
4,2012,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,0.634231,0.652453,4.867661e+08,3.189506e+09,2.708753e+10,...,13.984417,948.452874,1177.378233,22.349961,19.382833,84.829550,4.240343,0.783042,0.686846,1103.437000
5,2013,3.510722e+07,1.748211e+06,7.492332,2.846532e+07,0.637000,0.652741,5.225848e+08,3.093872e+09,2.523394e+10,...,13.914136,948.674218,1124.843955,22.352189,19.171814,83.798501,4.142761,0.783295,0.688000,1223.444000
6,2014,3.559137e+07,1.740643e+06,7.503408,2.852940e+07,0.641416,0.652552,5.676413e+08,2.779340e+09,1.954361e+10,...,13.565055,948.952159,1092.104700,22.396949,19.115911,83.444066,4.091372,0.783070,0.689500,1240.232857
7,2015,3.588415e+07,1.726042e+06,7.497228,2.851564e+07,0.643285,0.651015,5.179392e+08,2.561731e+09,1.327596e+10,...,13.574441,949.008244,1112.063713,22.423455,19.017059,82.914233,4.121849,0.782409,0.689458,1076.248286
8,2016,3.605286e+07,1.712480e+06,7.491294,2.849183e+07,0.655721,0.660706,4.524144e+08,2.516859e+09,1.011370e+10,...,13.373177,948.977114,1153.430368,22.490255,18.907433,81.942887,4.149759,0.794801,0.700800,928.236857
9,2017,3.609231e+07,1.709809e+06,7.490431,2.847638e+07,0.655605,0.660497,4.213689e+08,2.497710e+09,8.456309e+09,...,13.874302,948.990503,1103.061547,22.557983,18.973761,81.889312,4.064685,0.794147,0.700509,786.338000


In [31]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
0,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,0.618312,0.647756,5.483335e+08,2.546644e+09,1.874883e+10,2.351319e+10,...,8.076245,15.141120,948.241851,1293.296773,22.416524,19.527739,85.157286,4.537486,0.776272,0.677959
1,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,0.622639,0.649827,5.302764e+08,2.679945e+09,2.055015e+10,2.446518e+10,...,8.105416,15.401551,948.141401,1311.220354,22.466548,19.707229,85.814573,4.395830,0.778859,0.680849
2,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,0.626723,0.651346,5.046425e+08,2.891930e+09,2.342247e+10,2.606035e+10,...,8.069181,15.187968,948.153818,1296.766840,22.336126,19.669010,86.268192,4.409472,0.780851,0.683306
3,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,0.630566,0.652325,4.854622e+08,3.091988e+09,2.604265e+10,2.845738e+10,...,8.055989,14.484405,948.335984,1243.078346,22.321663,19.581222,85.940292,4.376316,0.782258,0.685344
4,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,0.634231,0.652453,4.867661e+08,3.189506e+09,2.708753e+10,3.181492e+10,...,10.160575,13.984417,948.452874,1177.378233,22.349961,19.382833,84.829550,4.240343,0.783042,0.686846
5,3.510722e+07,1.748211e+06,7.492332,2.846532e+07,0.637000,0.652741,5.225848e+08,3.093872e+09,2.523394e+10,3.629162e+10,...,10.474673,13.914136,948.674218,1124.843955,22.352189,19.171814,83.798501,4.142761,0.783295,0.688000
6,3.559137e+07,1.740643e+06,7.503408,2.852940e+07,0.641416,0.652552,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,10.857997,13.565055,948.952159,1092.104700,22.396949,19.115911,83.444066,4.091372,0.783070,0.689500
7,3.588415e+07,1.726042e+06,7.497228,2.851564e+07,0.643285,0.651015,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,11.087923,13.574441,949.008244,1112.063713,22.423455,19.017059,82.914233,4.121849,0.782409,0.689458
8,3.605286e+07,1.712480e+06,7.491294,2.849183e+07,0.655721,0.660706,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,11.207329,13.373177,948.977114,1153.430368,22.490255,18.907433,81.942887,4.149759,0.794801,0.700800
9,3.609231e+07,1.709809e+06,7.490431,2.847638e+07,0.655605,0.660497,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,11.081901,13.874302,948.990503,1103.061547,22.557983,18.973761,81.889312,4.064685,0.794147,0.700509


In [32]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      734.672000
1      834.169000
2      946.248000
3     1103.437000
4     1223.444000
5     1240.232857
6     1076.248286
7      928.236857
8      786.338000
9      707.565000
10     769.080000
11    1051.694000
12    1112.087000
13    1064.438000
14            NaN
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [33]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
0,-1.810146,-0.456113,-1.842947,-1.843442,-1.649508,-1.510728,1.204752,-0.990234,0.166091,-1.487840,...,-1.381612,1.277422,-1.184588,1.217879,-0.309664,0.928249,0.805107,1.556229,-1.490340,-1.593272
1,-1.664704,0.306639,-1.920915,-1.865820,-1.312007,-1.003354,0.820657,-0.546688,0.428911,-1.378556,...,-1.360179,1.672391,-1.481310,1.374327,0.143568,1.589532,1.219811,0.949694,-1.061748,-1.209465
2,-1.114834,1.074001,-1.063263,-1.080348,-0.993520,-0.631406,0.275392,0.158672,0.847995,-1.195437,...,-1.386802,1.348471,-1.444631,1.248168,-1.038098,1.448723,1.506013,1.008108,-0.731901,-0.883241
3,-0.592012,1.289389,-0.214200,-0.346889,-0.693862,-0.391424,-0.132596,0.824346,1.230291,-0.920269,...,-1.396494,0.281449,-0.906526,0.779540,-1.169137,1.125292,1.299130,0.866140,-0.498786,-0.612741
4,-0.110910,1.482858,0.447883,0.299042,-0.408085,-0.360153,-0.104860,1.148829,1.382743,-0.534838,...,0.149754,-0.476832,-0.561243,0.206068,-0.912742,0.394385,0.598328,0.283940,-0.368967,-0.413254
5,0.289216,0.877577,0.728876,0.686467,-0.192132,-0.289670,0.657046,0.830615,1.112297,-0.020933,...,0.380523,-0.583419,0.092593,-0.252485,-0.892562,-0.383058,-0.052194,-0.133878,-0.326982,-0.260055
6,0.626026,0.466529,1.037358,0.975815,0.152293,-0.335791,1.615453,-0.215960,0.282052,0.569384,...,0.662153,-1.112836,0.913611,-0.538254,-0.487021,-0.589019,-0.275819,-0.353913,-0.364315,-0.060896
7,0.829705,-0.326563,0.865248,0.913694,0.297969,-0.712364,0.558230,-0.940033,-0.632425,0.943972,...,0.831080,-1.098601,1.079279,-0.364039,-0.246874,-0.953211,-0.610107,-0.223421,-0.473776,-0.066429
8,0.947072,-1.063170,0.699968,0.806159,1.267810,1.661389,-0.835563,-1.089341,-1.093812,1.140087,...,0.918809,-1.403838,0.987324,-0.002965,0.358358,-1.357098,-1.222961,-0.103915,1.578876,1.439506
9,0.974517,-1.208274,0.675928,0.736391,1.258743,1.610294,-1.495939,-1.153056,-1.335633,1.102599,...,0.826656,-0.643832,1.026875,-0.442616,0.971987,-1.112732,-1.256763,-0.468181,1.470481,1.400886


In [34]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      734.672000
1      834.169000
2      946.248000
3     1103.437000
4     1223.444000
5     1240.232857
6     1076.248286
7      928.236857
8      786.338000
9      707.565000
10     769.080000
11    1051.694000
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [35]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
12,0.40411,-0.962374,-0.399635,-0.193099,0.993295,0.480333,1.108405,2.635823,-0.189959,0.726405,...,0.841228,1.184615,0.106438,-2.634304,1.662202,-0.324798,-0.839488,-2.471071,0.585058,0.817053


In [36]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1112.087
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [37]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [38]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [39]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[64199085, 1752510602, 1005578345, 2036526985, 3900613138, 1872322173, 3305884905, 3895401749, 3501400947, 4027620293, 2140626025, 1777234611, 2524078263, 4097467955, 2044335591, 1190164843, 1467987194, 3174961309, 3461216360, 3863723789, 2908615018, 2525744539, 1160211997, 1400265828, 3935390949, 1215719877, 1213649685, 3600430265, 4114910931, 3499322154, 2478546873, 2045607205, 3495959622, 3050704192, 3382740970, 1072673470, 1274189803, 3978164364, 3704801080, 4124027772, 3154294847, 2370058437, 2835717253, 621710453, 2333862646, 106088841, 1091576524, 2531460230, 2176774985, 3885466529, 3843012441, 1411154906, 3269985704, 3225792994, 3082212189, 4284522054, 393871212, 2350028629, 4009177737, 2886470399, 3062716696, 1659520041, 1798954965, 2769790462, 1072687542, 1425170160, 3377064999, 3342099723, 2268214314, 587792872, 2666586513, 318565823, 4270329086, 2440076495, 168105212, 1258551274, 1246183417, 1759946611, 1063289497, 2764833614, 2096822647, 2396530706, 4228505857, 2223449058,

2023-09-15 15:31:25.196849: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 15:31:25.229509: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 15:31:25.752938: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 15:31:25.787489: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 13566.51953125
winner_seed: 64199085


Step: 1 ___________________________________________
val_loss: 13851.884765625


Step: 2 ___________________________________________
val_loss: 17215.4453125


Step: 3 ___________________________________________
val_loss: 4265.73046875
winner_seed: 2036526985


Step: 4 ___________________________________________
val_loss: 34454.875


Step: 5 ___________________________________________
val_loss: 23673.533203125


Step: 6 ___________________________________________
val_loss: 7888.86376953125


Step: 7 ___________________________________________
val_loss: 26767.6875


Step: 8 ___________________________________________
val_loss: 22336.529296875


Step: 9 ___________________________________________
val_loss: 30977.50390625


Step: 10 ___________________________________________
val_loss: 27281.103515625


Step: 11 ___________________________________________
val_loss: 33377.82421875


Step: 12 ___________________________________________
val_loss

In [40]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 466ms/step - loss: 935499.0625 - val_loss: 1111212.5000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 921067.0000 - val_loss: 940082.9375
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 901295.1875 - val_loss: 1170391.7500
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 907553.6875 - val_loss: 743612.5625
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 904715.3125 - val_loss: 1155928.1250
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 898961.4375 - val_loss: 1201337.2500
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 862738.8125 - val_loss: 633737.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 802674.3125 - val_loss: 664321.8125
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 764563.5000 - val_loss: 652299.7500
Epoch

Epoch 75/10000
1/1 [==============================] - 0s 15ms/step - loss: 232539.3750 - val_loss: 278583.5938
Epoch 76/10000
1/1 [==============================] - 0s 15ms/step - loss: 227275.2500 - val_loss: 268499.1875
Epoch 77/10000
1/1 [==============================] - 0s 16ms/step - loss: 222848.9844 - val_loss: 265734.4688
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 218539.5469 - val_loss: 261571.0625
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 215165.8281 - val_loss: 259804.3750
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 211327.7969 - val_loss: 258211.9844
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 207306.2969 - val_loss: 254728.2031
Epoch 82/10000
1/1 [==============================] - 0s 15ms/step - loss: 203895.8750 - val_loss: 254166.5156
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 199674.9219 - val_loss: 249832.0000
E

1/1 [==============================] - 0s 14ms/step - loss: 71621.1250 - val_loss: 145882.0781
Epoch 149/10000
1/1 [==============================] - 0s 15ms/step - loss: 70677.2734 - val_loss: 144775.8438
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 69752.3516 - val_loss: 143684.5312
Epoch 151/10000
1/1 [==============================] - 0s 15ms/step - loss: 68845.9531 - val_loss: 142608.0156
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 67957.7266 - val_loss: 141546.0625
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 67087.3047 - val_loss: 140498.4062
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 66234.3203 - val_loss: 139465.0000
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 65398.4336 - val_loss: 138445.4062
Epoch 156/10000
1/1 [==============================] - 0s 15ms/step - loss: 64579.2812 - val_loss: 137439.6250
Epoch 157/10000
1

1/1 [==============================] - 0s 14ms/step - loss: 35033.4414 - val_loss: 93235.1406
Epoch 223/10000
1/1 [==============================] - 0s 19ms/step - loss: 34821.9219 - val_loss: 92825.0234
Epoch 224/10000
1/1 [==============================] - 0s 15ms/step - loss: 34614.6367 - val_loss: 92419.6406
Epoch 225/10000
1/1 [==============================] - 0s 17ms/step - loss: 34411.4727 - val_loss: 92018.3203
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 34212.3633 - val_loss: 91621.2734
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 34017.2305 - val_loss: 91228.3672
Epoch 228/10000
1/1 [==============================] - 0s 16ms/step - loss: 33825.9883 - val_loss: 90839.6172
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 33638.5586 - val_loss: 90455.1172
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 33454.8711 - val_loss: 90074.6875
Epoch 231/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 27469.0371 - val_loss: 75914.4062
Epoch 297/10000
1/1 [==============================] - 0s 15ms/step - loss: 26800.3301 - val_loss: 72467.7266
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 26726.7344 - val_loss: 72898.5859
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 26679.1484 - val_loss: 72546.2891
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 26633.5469 - val_loss: 72361.0000
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 26588.8848 - val_loss: 72147.4219
Epoch 302/10000
1/1 [==============================] - 0s 15ms/step - loss: 26545.1113 - val_loss: 71945.2578
Epoch 303/10000
1/1 [==============================] - 0s 15ms/step - loss: 26502.2031 - val_loss: 71746.0938
Epoch 304/10000
1/1 [==============================] - 0s 15ms/step - loss: 26460.1387 - val_loss: 71551.4531
Epoch 305/10000
1/1 [=====

...

1/1 [==============================] - 0s 15ms/step - loss: 25963.5859 - val_loss: 51229.1172
Epoch 1037/10000
1/1 [==============================] - 0s 15ms/step - loss: 25955.1484 - val_loss: 51288.0664
Epoch 1038/10000
1/1 [==============================] - 0s 15ms/step - loss: 25946.7871 - val_loss: 51343.7305
Epoch 1039/10000
1/1 [==============================] - 0s 19ms/step - loss: 25938.4941 - val_loss: 51396.3555
Epoch 1040/10000
1/1 [==============================] - 0s 15ms/step - loss: 25930.2734 - val_loss: 51445.9883
Epoch 1041/10000
1/1 [==============================] - 0s 15ms/step - loss: 25922.1191 - val_loss: 51492.8164
Epoch 1042/10000
1/1 [==============================] - 0s 15ms/step - loss: 25914.0293 - val_loss: 51537.0352
Epoch 1043/10000
1/1 [==============================] - 0s 18ms/step - loss: 25906.0000 - val_loss: 51578.7227
Epoch 1044/10000
1/1 [==============================] - 0s 16ms/step - loss: 25898.0293 - val_loss: 51618.0430
Epoch 1045/10000
1

Epoch 1110/10000
1/1 [==============================] - 0s 15ms/step - loss: 25462.7129 - val_loss: 52252.1445
Epoch 1111/10000
1/1 [==============================] - 0s 14ms/step - loss: 25457.1270 - val_loss: 52249.4648
Epoch 1112/10000
1/1 [==============================] - 0s 14ms/step - loss: 25451.5645 - val_loss: 52246.9531
Epoch 1113/10000
1/1 [==============================] - 0s 14ms/step - loss: 25446.0312 - val_loss: 52244.5547
Epoch 1114/10000
1/1 [==============================] - 0s 15ms/step - loss: 25440.5254 - val_loss: 52242.2109
Epoch 1115/10000
1/1 [==============================] - 0s 15ms/step - loss: 25435.0410 - val_loss: 52240.0898
Epoch 1116/10000
1/1 [==============================] - 0s 14ms/step - loss: 25429.5879 - val_loss: 52238.0000
Epoch 1117/10000
1/1 [==============================] - 0s 14ms/step - loss: 25424.1582 - val_loss: 52236.0195
Epoch 1118/10000
1/1 [==============================] - 0s 14ms/step - loss: 25418.7559 - val_loss: 52234.1758
E

Epoch 1184/10000
1/1 [==============================] - 0s 14ms/step - loss: 24797.0781 - val_loss: 57809.7812
Epoch 1185/10000
1/1 [==============================] - 0s 15ms/step - loss: 24790.1328 - val_loss: 57590.0977
Epoch 1186/10000
1/1 [==============================] - 0s 15ms/step - loss: 24783.5098 - val_loss: 57382.0039
Epoch 1187/10000
1/1 [==============================] - 0s 16ms/step - loss: 24777.1855 - val_loss: 57184.7344
Epoch 1188/10000
1/1 [==============================] - 0s 16ms/step - loss: 24771.1250 - val_loss: 56997.7734
Epoch 1189/10000
1/1 [==============================] - 0s 15ms/step - loss: 24765.3145 - val_loss: 56820.6328
Epoch 1190/10000
1/1 [==============================] - 0s 15ms/step - loss: 24759.7266 - val_loss: 56652.7148
Epoch 1191/10000
1/1 [==============================] - 0s 14ms/step - loss: 24754.3379 - val_loss: 56493.5469
Epoch 1192/10000
1/1 [==============================] - 0s 14ms/step - loss: 24749.1387 - val_loss: 56342.6875
E

Epoch 1258/10000
1/1 [==============================] - 0s 15ms/step - loss: 24539.3828 - val_loss: 53712.8984
Epoch 1259/10000
1/1 [==============================] - 0s 14ms/step - loss: 24537.0527 - val_loss: 53710.2109
Epoch 1260/10000
1/1 [==============================] - 0s 15ms/step - loss: 24534.7402 - val_loss: 53707.7812
Epoch 1261/10000
1/1 [==============================] - 0s 14ms/step - loss: 24532.4434 - val_loss: 53705.5430
Epoch 1262/10000
1/1 [==============================] - 0s 15ms/step - loss: 24530.1621 - val_loss: 53703.5352
Epoch 1263/10000
1/1 [==============================] - 0s 15ms/step - loss: 24527.8965 - val_loss: 53701.7539
Epoch 1264/10000
1/1 [==============================] - 0s 15ms/step - loss: 24525.6484 - val_loss: 53700.1133
Epoch 1265/10000
1/1 [==============================] - 0s 15ms/step - loss: 24523.4121 - val_loss: 53698.7266
Epoch 1266/10000
1/1 [==============================] - 0s 15ms/step - loss: 24521.1973 - val_loss: 53697.4531
E

Epoch 1332/10000
1/1 [==============================] - 0s 14ms/step - loss: 24404.4941 - val_loss: 53774.5352
Epoch 1333/10000
1/1 [==============================] - 0s 14ms/step - loss: 24403.1074 - val_loss: 53776.5156
Epoch 1334/10000
1/1 [==============================] - 0s 15ms/step - loss: 24401.7344 - val_loss: 53778.4688
Epoch 1335/10000
1/1 [==============================] - 0s 15ms/step - loss: 24400.3672 - val_loss: 53780.4258
Epoch 1336/10000
1/1 [==============================] - 0s 14ms/step - loss: 24399.0059 - val_loss: 53782.4336
Epoch 1337/10000
1/1 [==============================] - 0s 16ms/step - loss: 24397.6641 - val_loss: 53784.3594
Epoch 1338/10000
1/1 [==============================] - 0s 16ms/step - loss: 24396.3223 - val_loss: 53786.3398
Epoch 1339/10000
1/1 [==============================] - 0s 16ms/step - loss: 24394.9941 - val_loss: 53788.3242
Epoch 1340/10000
1/1 [==============================] - 0s 16ms/step - loss: 24393.6777 - val_loss: 53790.3047
E

In [41]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 35ms/step
[1112.087](test_target) - [[1060.1917]](prediction) = 51.89534960937499


In [42]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [43]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
0,-0.935924,-1.223513,-0.604041,-0.676239,-1.236405,-1.282559,1.148205,-1.122043,-1.121418,-1.109924,...,-0.469856,-0.903616,1.405096,-0.918766,0.188460,-1.385026,-1.292372,1.408816,-1.274302,-1.256477
1,-0.450207,-0.002459,-0.805386,-0.737532,0.023664,0.125255,0.140893,-0.184474,-0.185492,-0.204018,...,1.390102,1.393938,-0.841397,1.390459,1.119591,0.940056,0.148861,-0.811314,0.106005,0.066144
2,1.386131,1.225973,1.409428,1.413771,1.212741,1.157304,-1.289098,1.306518,1.306910,1.313942,...,-0.920245,-0.490322,-0.563699,-0.471692,-1.308051,0.444970,1.143510,-0.597501,1.168297,1.190332


train_target:


,Paraíba - Consumo de Cimento (t)
0,734.672
1,834.169
2,946.248


test_input:


...

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
12,0.40411,-0.962374,-0.399635,-0.193099,0.993295,0.480333,1.108405,2.635823,-0.189959,0.726405,...,0.841228,1.184615,0.106438,-2.634304,1.662202,-0.324798,-0.839488,-2.471071,0.585058,0.817053


test_target:


,Paraíba - Consumo de Cimento (t)
12,1112.087


1/1 [==============================] - 0s 37ms/step
Error: 51.89534960937499




[946.243408203125,
 1024.839599609375,
 1184.0992431640625,
 1240.2418212890625,
 1154.6767578125,
 953.6341552734375,
 785.1358642578125,
 704.0863037109375,
 768.917236328125,
 1060.191650390625]

In [44]:
display(targets)
display(predictions)

[1103.437,
 1223.444,
 1240.2328571428573,
 1076.248285714286,
 928.2368571428572,
 786.338,
 707.565,
 769.08,
 1051.694,
 1112.087]

[946.243408203125,
 1024.839599609375,
 1184.0992431640625,
 1240.2418212890625,
 1154.6767578125,
 953.6341552734375,
 785.1358642578125,
 704.0863037109375,
 768.917236328125,
 1060.191650390625]

In [45]:
mae = mean_absolute_error(predictions, targets)
mae

144.68978715122762

In [46]:
porcentage = mae/np.mean(targets)
porcentage

0.14471347674737117